In [4]:
# this file is to check and verify and report the result of the whole Bug Assignment:
import pandas as pd
import numpy as np
import math 
import statistics
from pandas import Timestamp
import math 

# Return the natural logarithm of different numbers


l1 = ['99329', '99329', '99199', '99072', '99072', '99072', '98679', '97858', '97529', '97529', '97474', '97474', '97468', '97397', '9737', '97316', '97313', '97313', '97313', '97313', '97201', '97201', '96949', '96949', '96917', '96799', '96581', '96183', '9615', '95937', '9587', '95709', '95619', '95619', '95619', '95528', '95148', '94810', '94794', '94789', '94789', '94665', '94652', '94652', '94652', '94652', '94652', '94643', '94242', '94170', '93964', '93964', '93567', '93552', '93105', '92751', '91266', '90930', '90144', '90016', '89526', '89526', '89523', '89517', '89500', '89500', '89015', '8815', '87258', '86842', '86602', '86441', '86152', '86152', '86107', '85299', '85299', '84670', '84670', '84533', '84070', '82992', '82311', '82311', '82311', '82311', '82311', '82311', '82061', '81983', '8142', '81177', '81146', '81131', '81098', '8085', '80393', '78867', '7882', '77901', '77612', '77596', '77596', '77030', '77030', '76348', '76348', '76158', '76158', '76158', '76099', '75800', '75794', '75711', '75711', '75432', '75264', '74647', '71153', '70728', '70158', '6794', '67691', '67417', '67124', '67046', '66646', '65246', '64418', '64105', '63845', '63400', '63145', '62878', '62386', '6146', '6125', '60557', '60349', '58293', '57011', '55899', '55899', '55845', '5264', '52257', '5183', '51704', '51619', '51466', '51451', '51187', '50658', '49177', '49002', '48970', '48482', '48393', '47874', '46933', '46042', '45438', '45438', '45438', '45347', '44721', '44419', '44166', '43766', '4358', '43003', '42785', '42639', '42587', '42587', '42587', '42587', '41727', '41489', '40780', '40360', '40047', '38955', '38503', '38491', '37249', '37149', '36578', '36514', '36486', '35107', '34685', '34067', '33737', '33648', '33648', '33535', '33230', '33163', '31957', '31957', '31307', '31307', '31088', '30970', '30706', '30700', '30700', '30700', '30683', '29726', '28875', '287374', '287374', '285854', '285854', '285814', '28547', '28547', '28547', '28547', '28547', '285437', '284793', '284793', '284793', '280123', '279981', '277299', '277299', '277299', '277299', '27687', '27512', '27483', '274474', '274473', '27429', '274091', '274091', '27408', '274041', '272742', '27264', '271375', '269401', '269401', '269167', '267616', '267616', '267178', '266938', '26611', '266092', '266092', '265448', '265197', '264770', '264143', '264120', '26295', '262909', '26252', '26250', '26242', '261100', '260626', '260626', '260626', '259413', '259413', '259413', '25903', '256577', '254721', '253674', '252104', '252104', '251824', '251789', '251789', '251789', '251771', '25129', '250960', '250944', '250663', '250663', '250663', '250663', '250660', '249535', '24890', '24890', '247012', '24678', '24674', '246627', '24571', '244020', '244020', '244020', '243912', '243584', '242762', '241193', '241193', '240690', '240690', '240690', '240690', '239695', '239695', '239695', '239647', '239647', '239647', '239647', '238718', '238353', '238353', '238218', '238218', '238218', '238218', '238218', '238218', '238218', '236878', '236224', '234361', '234163', '233954', '233559', '232980', '232773', '232246', '232025', '231575', '230391', '229302', '229261', '229031', '226894', '226195', '226132', '224132', '223818', '222395', '222369', '221782', '221767', '221743', '220257', '220124', '220013', '220002', '220002', '220002', '220002', '220002', '220002', '219405', '219405', '21904', '21857', '218431', '218339', '218339', '217819', '216560', '216314', '215161', '214340', '213519', '213248', '212545', '212015', '210883', '210388', '210388', '209778', '209505', '209221', '209220', '209220', '208066', '208066', '208066', '208066', '207946', '207839', '207839', '20644', '206153', '206153', '205908', '205177', '204682', '204682', '204682', '204137', '203256', '201590', '201096', '201063', '20047', '199376', '199215', '19864', '19859', '19859', '197791', '197768', '197481', '196334', '196075', '196075', '195808', '19431', '191804', '19145', '19145', '191222', '191208', '19066', '189987', '188728', '188728', '188728', '188728', '188728', '188728', '188130', '18780', '187004', '185482', '185482', '185401', '185332', '185332', '185177', '185177', '183857', '182725', '182341', '181788', '18082', '180406', '179473', '17916', '179022', '178744', '178744', '178744', '178503', '177749', '177749', '177749', '177749', '177749', '177749', '177749', '177749', '177749', '176837', '17667', '175495', '175495', '17383', '173012', '172516', '171525', '171525', '171525', '17143', '169840', '16938', '16822', '167682', '167622', '167616', '167472', '164324', '163806', '163495', '163121', '163121', '163121', '162573', '162530', '16230', '16230', '159157', '15906', '157812', '15656', '154103', '152176', '152142', '152004', '151988', '151988', '151611', '15152', '151404', '150249', '14898', '147781', '14299', '141994', '14093', '140416', '140392', '140332', '140111', '139818', '139805', '138320', '13799', '135053', '134536', '13243', '13170', '131279', '129725', '129168', '12884', '127780', '127678', '127563', '126616', '126616', '12590', '125677', '125450', '123985', '123356', '123234', '122319', '122176', '121979', '121084', '121033', '12101', '120864', '120598', '120598', '120340', '119170', '119170', '118152', '117894', '11767', '11763', '116848', '116223', '116199', '115860', '115860', '115860', '115860', '115526', '11546', '11546', '115016', '115016', '114558', '114499', '11442', '11442', '113998', '113841', '113841', '113841', '112586', '111441', '110015', '110015', '110003', '10970', '108666', '107637', '107637', '107621', '107128', '107128', '106855', '106174', '104379', '103078', '102494', '10247', '10247', '102281', '102018', '101233', '101210', '101210', '100539']
l2 = ['96799', '93964', '64105', '62386', '55899', '51704', '49177', '42639', '33737', '30700', '27483', '27408', '27264', '265448', '243584', '239647', '238718', '221767', '21904', '212545', '18082', '178744', '178744', '177749', '159157', '157812', '129725', '127678', '121033', '116223']

final_list = []
for data in l2:
    if l1.count(data) == 1:
        final_list.append(data)

print(final_list)

'93964', '55899', '30700', '239647', '178744', '178744', '177749'

['96799', '64105', '62386', '51704', '49177', '42639', '33737', '27483', '27408', '27264', '265448', '243584', '238718', '221767', '21904', '212545', '18082', '159157', '157812', '129725', '127678', '121033', '116223']


('93964', '55899', '30700', '239647', '178744', '178744', '177749')

In [81]:
# multiply instead of the summation
def multiplyList(myList) :
    result = 1
    for x in myList:
         result = result * x
    return result

# get stat for -old or new and then compare
def get_stat_for(file, authors, label='-new'):
    pd.set_option("display.max_columns", None)
    df = pd.read_csv(file)
    
    df1 = pd.read_csv('../../data/output/del2/jdt_direct-super-indirect.csv')
    
    
    result = df.loc[[385, 130, 132, 261, 271, 147, 277, 158, 31, 164, 50, 437, 182, 443, 576, 454, 459, 461, 220, 94, 97, 98, 363, 492, 110, 111, 241, 371, 244, 374, 377, 509]]
    result.to_csv('fixable2.csv')  
    
    
    count = 0
    total = 0

    previous_component = ''
    previous_time = ''
    previous = ''
    ids = set()
    for (index, row), (index_s, row_s) in zip(df.iterrows(), df1.iterrows()):
        #print(row['bug_id'] == row_s['bug_id'])
        diff = Timestamp(row['report_time']) - Timestamp(previous_time)
        hdiff = diff.total_seconds() / 3600
        
        if False and previous != '' and hdiff < 2:
            chosen = previous
        else:
            total_scores = {'history': 0, 'api': 0, 'code': 0, 'api_s': 0}
            for i in range(1, 11):
                h = row['history_at_' + str(i) + '_v']
                c = row['code_at_' + str(i) + '_v']
                a = row['api_at_' + str(i) + '_v']

                if str(h) != 'nan':
                    total_scores['history'] += h
                if str(c) != 'nan':
                    total_scores['code'] += c
                if str(a) != 'nan':
                    total_scores['api'] += a
               
                a_s = row_s['api_at_' + str(i) + '_v']
                if str(a) != 'nan':
                    total_scores['api_s'] += a_s
                    
                    
            scores = {}
            for i in range(1, 11):
                h = row['history_at_' + str(i) + '_v']
                h_developer = row['history_at_' + str(i)]
                c = row['code_at_' + str(i) + '_v']
                c_developer = row['code_at_' + str(i)]
                a = row['api_at_' + str(i) + '_v']
                a_developer = row['api_at_' + str(i)]

                a_s = row_s['api_at_' + str(i) + '_v']
                a_s_developer = row_s['api_at_' + str(i)]
                
                if str(h_developer) != 'nan':
                    if str(h) == 'nan' or h == 0 or total_scores['history'] <= 0:
                        scores[a_developer] = scores.get(a_developer, []) + [0]
                    else:
                        scores[h_developer] = scores.get(h_developer, []) + [float(h * 100 / total_scores['history'])]

                if str(c_developer) != 'nan':
                    if str(c) == 'nan' or c == 0 or total_scores['code'] <= 0:
                        scores[a_developer] = scores.get(a_developer, []) + [0]
                    else:
                        scores[c_developer] = scores.get(c_developer, []) + [ float(c * 100 / total_scores['code'])]


                if str(a_developer) != 'nan':
                    if str(a) == 'nan' or a == 0 or total_scores['api'] <= 0:
                        scores[a_developer] = scores.get(a_developer, []) + [0]
                    else:
                        scores[a_developer] = scores.get(a_developer, []) + [row['jaccard_score'] * float(a * 100 / total_scores['api'])]
                        pass

            for index_, score in scores.items():
                if len(score) == 0:
                    scores[index_] = 0
                else:
                    scores[index_] = sum(score)  ## ** (1/float(len(score)))


#             diff = Timestamp(row['report_time']) - Timestamp(previous_time)
#             hdiff = diff.total_seconds() / 3600
#             if hdiff < 0.1 and row['component'] == previous_component:
#                 scores[previous] = 0
            # sorting the results
            sort = sorted(scores.items(), key=lambda x: x[1], reverse=True)
            chosen = ''
            if len(sort) != 0:
                chosen = sort[0][0]

        for s in sort:
            if s[1] == sort[0][1]:
                if row['assignees'] == s[0]:
                    count += 1
                    break
            break
                
            

        if chosen == row['author']  and chosen != row['assignees']:
            ids.add(row['bug_id'])
            
            
        previous = row['assignees']
        total += 1

        if row['assignees'] not in authors:
            authors[row['assignees']] = {
                'total-new': 0, 'count_differently-new': 0, 'count-new': 0, 'count_history-new': 0, 'count_code-new': 0, 'count_api-new': 0,
                'total-old': 0, 'count_differently-old': 0, 'count-old': 0, 'count_history-old': 0, 'count_code-old': 0, 'count_api-old': 0,
                }

        authors[row['assignees']]['total' + label] += 1
        if row['assignees'] == row['history_at_1']:
            authors[row['assignees']]['count_history' + label] += 1
        if row['assignees'] == row['code_at_1']:
            authors[row['assignees']]['count_code' + label] += 1
        if row['assignees'] == row['api_at_1']:
            authors[row['assignees']]['count_api' + label] += 1
        if 1 == row['at_1']:
            authors[row['assignees']]['count' + label] += 1
        if 0 < len(sort) and sort[0][0] == row['assignees']:
            authors[row['assignees']]['count_differently' + label] += 1
            
#         if 0 < len(sort) and row['at_1'] == 1 and 53 < sort[0][0] < 107:
#             sort[0][0]
            
            
        if 0 < len(sort) and row['at_1'] == 0 and sort[0][0] == row['assignees']:
            print('dd')
            print(row['component'])
        
        previous = row['chosen']
        previous_time = row['report_time']
        previous_component = row['component']
   
    print(len(ids))
    print((ids))
    print(label)
    print(count)
    print(count / total)
    print(total)
    return authors



authors_ = {}
authors_ = get_stat_for('../../data/output/jdt_direct-46.csv', authors_, '-new')
#authors_ = get_stat_for('../../data/output/jdt_direct-without-util.csv', authors_, '-old')

16
{289280, 295040, 313313, 328641, 421506, 305255, 319532, 295629, 296878, 314094, 306736, 331311, 299155, 339700, 320511, 304927}
-new
276
0.46
600


In [82]:
a= 0
authors_copy = {}
for name, author in authors_.items():
    author['count_differently-change'] = author['count_differently-new'] - author['count_differently-old']
    author['count-change'] = author['count-new'] - author['count-old']
    #author['history-change'] = author['count_history-new'] - author['count_history-old']
    #author['code-change'] = author['count_code-new'] - author['count_code-old']
    author['api-change'] = author['count_api-new'] - author['count_api-old']
    #author['can-be'] = author['total-new'] - author['count_differently-new']
    a = a + author['count-change']
    
    authors_copy[name] = {'api': author['count_api-new'], 'total': author['total-new'], 'count': author['count_differently-new']}

print(a)
x = pd.DataFrame.from_dict(authors_copy)
x

276


,Dani Megert,Markus Keller,Satyam Kandula,Kent Johnson,JDT-UI-Inbox,Olivier Thomann,Deepak Azad,Frederic Fusier,JDT-Text-Inbox,Raksha Vasisht,Jayaprakash Arthanareeswaran,Paul Webster,Rajesh,Tom Hofmann,Lakshmi Shanmugam,Björn Michael,Manju Mathew,Noopur Gupta,Platform-Search-Inbox,Paul Fullbright,Ayushman Jain,Platform-Doc-Inbox,Nikolay Metchev,Szymon Brandys,Samrat Dhillon,Philippe Marschall,Stephan Herrmann,Rabea Gransberger,Manoj Palat,Snjezana Peco,Michael Rennie,Lars Vogel,Andrew Eisenberg,shankha banerjee,Marcel Bruch
api,46,169,0,0,0,0,5,0,0,1,0,0,0,0,0,0,2,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
total,145,241,2,1,8,2,85,1,6,37,2,1,7,1,1,2,18,12,1,1,2,1,3,1,5,1,4,1,1,1,2,1,1,1,1
count,79,180,0,0,0,0,10,0,0,3,0,0,0,0,0,0,2,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [120]:
components = {}
a = 0
df = pd.read_csv('../../data/output/jdt_direct-46.csv')
for index, row in df.iterrows():
    if row['component'] not in components:
        components[row['component']] = {'count': 0, 'total': 0}
    
    if row['at_1'] == 1:
        a += 1
    if row['at_1'] == 1:
        components[row['component']]['count'] += 1
    components[row['component']]['total'] += 1

    
print(a) 
x = pd.DataFrame.from_dict(components)
x

276


,JDT-Text,JDT-UI,JDT-APT,JDT-Core,Platform-Text,Platform-Search,Platform-UI,Platform-IDE,Platform-SWT,Platform-Doc,Platform-Resources,PDE-API Tools
count,71,198,0,0,3,1,2,0,0,1,0,0
total,160,407,1,11,10,2,4,1,1,1,1,1


In [121]:
components = {}
a = 0
df = pd.read_csv('../../data/output/jdt_direct.csv')
for index, row in df.iterrows():
    if row['component'] not in components:
        components[row['component']] = {'count': 0, 'total': 0}
    
    if row['at_1'] == 1:
        a += 1
    if row['at_1'] == 1:
        components[row['component']]['count'] += 1
    components[row['component']]['total'] += 1

    
print(a) 
x = pd.DataFrame.from_dict(components)
x

267


,JDT-Text,JDT-UI,JDT-APT,JDT-Core
count,71,196,0,0
total,160,407,1,11


In [108]:
a= 0
authors_copy = {}
for name, author in authors_.items():
    author['count_differently-change'] = author['count_differently-new'] - author['count_differently-old']
    author['count-change'] = author['count-new'] - author['count-old']
    #author['history-change'] = author['count_history-new'] - author['count_history-old']
    #author['code-change'] = author['count_code-new'] - author['count_code-old']
    author['api-change'] = author['count_api-new'] - author['count_api-old']
    #author['can-be'] = author['total-new'] - author['count_differently-new']
    a = a + author['count-change']
    
    authors_copy[name] = {'api': author['count_api-new'], 'total': author['total-new'], 'count': author['count_differently-new']}

print(a)
x = pd.DataFrame.from_dict(authors_copy)
x

266


,Dani Megert,Markus Keller,Satyam Kandula,Kent Johnson,JDT-UI-Inbox,Olivier Thomann,Deepak Azad,Frederic Fusier,JDT-Text-Inbox,Raksha Vasisht,Jayaprakash Arthanareeswaran,Paul Webster,Rajesh,Tom Hofmann,Lakshmi Shanmugam,Björn Michael,Manju Mathew,Noopur Gupta,Platform-Search-Inbox,Paul Fullbright,Ayushman Jain,Platform-Doc-Inbox,Nikolay Metchev,Szymon Brandys,Samrat Dhillon,Philippe Marschall,Stephan Herrmann,Rabea Gransberger,Manoj Palat,Snjezana Peco,Michael Rennie,Lars Vogel,Andrew Eisenberg,shankha banerjee,Marcel Bruch
api,39,96,0,0,0,0,5,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
total,145,241,2,1,8,2,85,1,6,37,2,1,7,1,1,2,18,12,1,1,2,1,3,1,5,1,4,1,1,1,2,1,1,1,1
count,39,96,0,0,0,0,5,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [19]:
x = pd.DataFrame.from_dict(authors_)
x

,Dani Megert,Markus Keller,Satyam Kandula,Kent Johnson,JDT-UI-Inbox,Olivier Thomann,Deepak Azad,Frederic Fusier,JDT-Text-Inbox,Raksha Vasisht,Jayaprakash Arthanareeswaran,Paul Webster,Rajesh,Tom Hofmann,Lakshmi Shanmugam,Björn Michael,Manju Mathew,Noopur Gupta,Platform-Search-Inbox,Paul Fullbright,Ayushman Jain,Platform-Doc-Inbox,Nikolay Metchev,Szymon Brandys,Samrat Dhillon,Philippe Marschall,Stephan Herrmann,Rabea Gransberger,Manoj Palat,Snjezana Peco,Michael Rennie,Lars Vogel,Andrew Eisenberg,shankha banerjee,Marcel Bruch
total-new,145,241,2,1,8,2,85,1,6,37,2,1,7,1,1,2,18,12,1,1,2,1,3,1,5,1,4,1,1,1,2,1,1,1,1
count_differently-new,77,173,0,0,0,0,10,0,0,3,0,0,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
count-new,77,173,0,0,0,0,10,0,0,3,0,0,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
count_history-new,36,107,0,0,0,0,2,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
count_code-new,75,146,0,0,0,0,20,0,0,6,0,0,0,0,0,0,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
count_api-new,6,18,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
total-old,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
count_differently-old,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
count-old,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
count_history-old,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
